In [ ]:
import random
import math
import pandas as pd
from category_encoders import TargetEncoder
from category_encoders import OneHotEncoder
from category_encoders import CountEncoder
from category_encoders import BinaryEncoder
df=pd.read_csv("allegro-api-transactions.csv")
import warnings
warnings.filterwarnings('ignore')

# Zadanie domowe 2

Zajmiemy się przekształceniem bazy allegro w pierwszej części konując jej zmienne kategoryczne, a w drugiej uzupełniając brakujące dane.

## Część pierwsza

Rozpoczniemy od zakodowania zmiennej kategorycznej *it_location* za pomocą target encoding.

In [ ]:
df['it_location'].describe()

Zauważamy, że kolumna ta przyjmuje 10056 unikalnych wartości, co świadczy o tym, że metoda *one hot encoding* nie będzie tu najlepszym pomysłem, bowiem stworzyłaby ona włąśnie tyle kolumn. Natomiast *target encoding* pozwoli zmniejszyć ilość zmiennych.

In [ ]:
te=TargetEncoder()
df1=df.copy()
df1['it_location_encoded']=te.fit_transform(df1['it_location'],df1['price'])
df1.head()

Stworzyliśmy w ramce danych nową kolumnę *it_location_encoded*, w której umieszczone są zakodowane wartości *it_location* za pomocą *target encoding*.

*Target encoding* polega na kodowaniu zmiennej kategorycznej jako: średniej wartości targetu(tu *price*), dla danej kategorii.

Zalety:
   - prosty i szybki
   - nie zwiększa wymiaru bazy danych 

Wady:
   - jest zależny od rozkładu targetu, co oznacza, że ma skłonności do overfitting-u
   - jest specyficzny co do danych i żadko pokazuje znaczącą poprawę

In [ ]:
ohe=OneHotEncoder(use_cat_names=True)
df2=df.copy()
df2=df2.join(ohe.fit_transform(df2.main_category))
df2.head()

Stworzyliśmy w ramce danych nowe kolumny, które kodują wartości *main_category* za pomocą one hot encoding. Dokonuje tego poprzez stworzenie po jednej kolumnie dla każdej unikalnej wartości w kolumnie *main_category*

Zalety:
   - działa dobrze z nominalnymi danymi

Wady:
   - może stworzyć naprawdę duże ramki danych

In [ ]:
be=BinaryEncoder()
df3=df.copy()
df3=df3.join(be.fit_transform(df3.main_category))
df3.head()

Stworzyliśmy w ramce danych nowe kolumny, które kodują wartości *main_category* za pomocą *binary encoding*. Działa ono podobnie jak *one hot*, ale przechowuje wartości jako binarne bitstring-i.

Co prawda metoda ta nie tworzy tak wielu kolumn jak one hot, ale przypomina przez to zbytnio *ordinal encoder* tyle, że w postaci binarnej.

In [ ]:
ce=CountEncoder()
df4=df.copy()
df4['main_category_encoded']=ce.fit_transform(df4.main_category)
df4.head()

Stworzyliśmy w ramce danych nowe kolumny, które kodują wartości *main_category* za pomocą *count encoding*. Dokonuje tego zamieniając każdą wartość kategoryczną ilością jej wystąpień.

Metoda ta może powodować wiele problemów np. gdy kolumna zawiera tylko dwie wartości kategoryczne i każda z nich występuje dokładnie tyle samo razy.

## Część druga

Rozpoczniemy od ograniczenia bazy danych do zmiennych numerycznych.

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import mean_squared_error
import seaborn as sns
import numpy as np

In [ ]:
def seller_rating():
    answer1=[0 for i in range(10)]
    answer=pd.DataFrame()
    for i in range(10):
        copy=df.loc[:,['price', 'it_seller_rating', 'it_quantity']].copy()
        random_sample1=random.sample(range(0,df.shape[0]),math.floor(df.shape[0]*0.1))
        copy.iloc[random_sample1,1]=None

        imp=IterativeImputer(max_iter=10,random_state=21)
        ans=pd.DataFrame(imp.fit_transform(copy),columns=['price', 'it_seller_rating', 'it_quantity'])
        answer1[i] = math.sqrt(mean_squared_error(df['it_seller_rating'], ans['it_seller_rating']))

    answer['it_seller_rating']=answer1
    return answer

In [ ]:
def both():
    answer1=[0 for i in range(10)]
    answer2=[0 for i in range(10)]
    answer=pd.DataFrame()
    for i in range(10):
        copy=df.loc[:,['price', 'it_seller_rating', 'it_quantity']].copy()
        random_sample1=random.sample(range(0,df.shape[0]),math.floor(df.shape[0]*0.1))
        random_sample2=random.sample(range(0,df.shape[0]),math.floor(df.shape[0]*0.1))
        copy.iloc[random_sample1,1]=None
        copy.iloc[random_sample2,2]=None
        imp=IterativeImputer(max_iter=10,random_state=21)
        ans=pd.DataFrame(imp.fit_transform(copy),columns=['price', 'it_seller_rating', 'it_quantity'])
        answer1[i] = math.sqrt(mean_squared_error(df['it_seller_rating'], ans['it_seller_rating']))
        answer2[i]=math.sqrt(mean_squared_error(df['it_quantity'], ans['it_quantity']))
    answer['it_seller_rating']=answer1
    answer['it_quantity']=answer2
    return answer

In [ ]:
answer1=seller_rating()

In [ ]:
answer=both()

In [ ]:
plotdata=pd.DataFrame()
plotdata['one_variable_missing']=answer1['it_seller_rating']
plotdata['two_variables_missing']=answer['it_seller_rating']
sns.boxplot(data=pd.melt(plotdata),x='variable', y='value')

Z powyższego wykresu łatwo wywnioskować, że algorytm gorzej działa, gdy brakuje więcej niż jednego argumentu.